In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from pprint import pprint

In [3]:
data = pd.read_csv('train_new.csv').sample(frac=1) #shuffle
data = data.loc[data['rating'].isin([1, 2, 3, 4, 5])]
data = data.fillna(0)
data = data.drop(['merchant_id', 'merchant_profile_picture', 'id', 'tags'], axis=1)

In [4]:
msk = np.random.rand(len(data)) < 0.7
tr = data[msk]
val = data[~msk]

In [10]:
tr

,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating,rating_count,badges_count,badge_local_product,badge_product_quality,...,urgency_text,origin_country,merchant_title,merchant_name,merchant_info_subtitle,merchant_rating_count,merchant_rating,merchant_has_profile_picture,theme,crawl_month
632,6.00,33,0,10,0,2.0,3,0,0,0,...,0,1,450,569,287,32394,3.924677,0,0,0
631,9.00,135,0,100,0,4.0,33,0,0,0,...,0,1,354,204,136,9940,4.048994,0,0,0
588,11.00,11,0,5000,1,4.0,1329,0,0,0,...,1,1,134,163,494,246312,4.203725,0,0,0
624,11.00,10,0,100,1,4.0,25,2,1,1,...,1,1,170,361,570,794,4.277078,0,0,0
1079,7.00,50,0,1000,1,4.0,408,0,0,0,...,1,1,392,257,302,12763,3.967484,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,6.00,7,0,50,0,3.0,3,0,0,0,...,0,1,482,432,346,1374,4.058952,0,0,0
455,2.76,17,0,1000,1,5.0,49,1,0,1,...,0,1,181,391,440,11505,4.136723,0,0,0
877,16.00,111,0,1000,1,5.0,580,0,0,0,...,0,1,346,188,452,20387,4.193457,0,0,0
740,5.88,9,0,20000,0,4.0,6450,0,0,0,...,0,1,171,362,17,12304,4.319002,0,0,0


In [5]:
dict_cat = {}


# columns that are of categorical value
cat_cols = tr.columns[tr.dtypes==object].to_list()



def cat_digit(col):  
    # build the mapping
    encoded = col.astype('category').cat.codes
    # store the mapping
    dict_cat[col.name] = dict(zip(np.asarray(col), np.asarray(encoded)))
    return encoded

# for each categorical feature, apply cat_digit where we build the mapping and transform the data
# this is for the training set (where we build the mapping)
tr[cat_cols] = tr[cat_cols].apply(lambda col: cat_digit(col))
tr

/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating,rating_count,badges_count,badge_local_product,badge_product_quality,...,urgency_text,origin_country,merchant_title,merchant_name,merchant_info_subtitle,merchant_rating_count,merchant_rating,merchant_has_profile_picture,theme,crawl_month
632,6.00,33,0,10,0,2.0,3,0,0,0,...,0,1,450,569,287,32394,3.924677,0,0,0
631,9.00,135,0,100,0,4.0,33,0,0,0,...,0,1,354,204,136,9940,4.048994,0,0,0
588,11.00,11,0,5000,1,4.0,1329,0,0,0,...,1,1,134,163,494,246312,4.203725,0,0,0
624,11.00,10,0,100,1,4.0,25,2,1,1,...,1,1,170,361,570,794,4.277078,0,0,0
1079,7.00,50,0,1000,1,4.0,408,0,0,0,...,1,1,392,257,302,12763,3.967484,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,6.00,7,0,50,0,3.0,3,0,0,0,...,0,1,482,432,346,1374,4.058952,0,0,0
455,2.76,17,0,1000,1,5.0,49,1,0,1,...,0,1,181,391,440,11505,4.136723,0,0,0
877,16.00,111,0,1000,1,5.0,580,0,0,0,...,0,1,346,188,452,20387,4.193457,0,0,0
740,5.88,9,0,20000,0,4.0,6450,0,0,0,...,0,1,171,362,17,12304,4.319002,0,0,0


In [6]:
print('categorical features')
pprint(list(dict_cat.keys()))

categorical features
['currency_buyer',
 'product_color',
 'product_variation_size_id',
 'shipping_option_name',
 'urgency_text',
 'origin_country',
 'merchant_title',
 'merchant_name',
 'merchant_info_subtitle',
 'theme',
 'crawl_month']


In [7]:
print('Lets see what the mapping for column origin_country :')
pprint(dict_cat['origin_country'])
print('It is a string to integer mapping')

Lets see what the mapping for column origin_country :
{0: 0, 'CN': 1, 'GB': 2, 'SG': 3, 'US': 4, 'VE': 5}
It is a string to integer mapping


In [9]:
# then we will use the mappings built from the training set, to transform the validation set
val[cat_cols] = val[cat_cols].apply(lambda col: col.map(dict_cat[col.name]))
# for string values that not seen in training set, we replace it with -1
val = val.fillna(-1)
val

,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating,rating_count,badges_count,badge_local_product,badge_product_quality,...,urgency_text,origin_country,merchant_title,merchant_name,merchant_info_subtitle,merchant_rating_count,merchant_rating,merchant_has_profile_picture,theme,crawl_month
981,8.0,7,-1.0,10000,0,4.0,2543,0,0,0,...,0.0,-1.0,-1.0,-1.0,-1.0,110548,4.143666,1,-1.0,-1.0
975,9.0,8,-1.0,5000,1,4.0,827,0,0,0,...,0.0,-1.0,-1.0,-1.0,-1.0,14275,4.091419,0,-1.0,-1.0
185,6.0,21,-1.0,5000,0,3.0,573,0,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,9827,4.113870,1,-1.0,-1.0
638,9.0,34,-1.0,100,1,3.0,28,0,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1472,3.534647,0,-1.0,-1.0
779,9.0,9,-1.0,1000,0,4.0,216,0,0,0,...,0.0,-1.0,-1.0,-1.0,-1.0,4298,4.229409,0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,11.0,68,-1.0,1000,0,4.0,211,0,0,0,...,0.0,-1.0,-1.0,-1.0,-1.0,11597,4.215228,0,-1.0,-1.0
989,16.0,14,-1.0,10,0,3.0,3,0,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,31,3.548387,0,-1.0,-1.0
599,11.0,10,-1.0,100,1,4.0,50,0,0,0,...,0.0,-1.0,-1.0,-1.0,-1.0,225438,4.301120,0,-1.0,-1.0
2,6.0,34,-1.0,100,0,4.0,10,0,0,0,...,0.0,-1.0,-1.0,-1.0,-1.0,3897,3.928920,0,-1.0,-1.0


In [11]:
tr_y = tr['rating']
tr_x = tr.drop('rating', axis=1)
clf = LogisticRegression().fit(tr_x, tr_y)

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
val_y = val['rating']
val_x = val.drop('rating', axis=1)
pred_val = clf.predict(val_x)

In [13]:
val_score = f1_score(val_y, pred_val, average='micro')
print(val_score)

0.696319018404908


In [14]:
# once you are happy with your local model, let's prepare a submission
# we need to apply the same preprocessing steps on the testing set as you did before you train the model

test_data = pd.read_csv('test_new.csv').sample(frac=1) 
_id = test_data['id']
test_data = test_data.fillna(0)
test_data = test_data.drop(['merchant_id', 'merchant_profile_picture', 'id', 'tags'], axis=1)
test_data[cat_cols] = test_data[cat_cols].apply(lambda col: col.map(dict_cat[col.name]))

# again, not-seen string value filled with -1
test_data = test_data.fillna(-1)

In [15]:
pred_test = clf.predict(test_data)
pred_df = pd.DataFrame(data={'id': np.asarray(_id), 'rating': pred_test})
pred_df.to_csv('pred_walkthrough.csv', index=False)